In [1]:
import pandas as pd
import csv
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Lambda, LSTM, Dense, Dropout, Input
import os
import re
import pickle
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from numba import cuda 

In [2]:
print(tf.test.is_gpu_available())
print(tf.config.list_physical_devices('GPU'))

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
def clear_GPU():
    device = cuda.get_current_device()
    device.reset()

In [4]:
MODE_PDUMP = False

## Data Loading from lyrics_l.pkl

In [5]:
fin_fil_df = pd.read_pickle('pickles/fin.pkl')

In [19]:
if MODE_PDUMP:
    directory_path = 'data/music4all_subset/lyrics'
    file_prefixes = fin_fil_df['id'].tolist()
    lyrics_d = {}
    for prefix in file_prefixes:
        file_pattern = prefix + '.txt'
        file_path = os.path.join(directory_path, file_pattern)
        if os.path.exists(file_path):
            with open(file_path, 'r', encoding="utf8") as file:
                content = file.read()
                lyrics_d[prefix] = content
                lyrics_l.append(content)
                # print(f"Content of {file_pattern}:\n{content}")
    
    with open('pickles/lyrics_l.pkl', 'wb') as f:
        pickle.dump(lyrics_l, f)

In [8]:
with open('pickles/lyrics_l.pkl', 'rb') as f:
    lyrics_l = pickle.load(f)

## Preprocessing

In [9]:
def preprocess_lyrics(lyli):
    def prep_single(ly):
        text = re.sub(r'[^\w\s]', '', ly).lower()
        text = text.replace('\n', ' ')
        text = re.sub(' +', ' ', text)
        return text
    new = [prep_single(l) for l in lyli]
    return new

In [10]:
prep_lyrics = preprocess_lyrics(lyrics_l)

In [11]:
encoder = LabelEncoder()
labels = fin_fil_df['genre']
genre_labels = encoder.fit_transform(labels)
print(encoder.classes_)
X_train, X_test, y_train, y_test = train_test_split(prep_lyrics, genre_labels, test_size=0.2)

['alternative rock' 'ambient' 'classic rock' 'electronic' 'folk'
 'indie rock' 'metal' 'pop' 'rap' 'singer-songwriter' 'soul']


## Lyric Models

In [14]:
es_cb = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
def getModelCheckpoint(name):
    return ModelCheckpoint(
        filepath=f'models/{name}.h5',
        save_best_only=True,
        monitor='val_accuracy',
        mode='max',
        verbose=1
    )

### ELMo

In [15]:
elmo = hub.load('https://tfhub.dev/google/elmo/2').signatures['default']

In [135]:
def elmo_vectors(x):
  embeddings = elmo(tf.constant(x))["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings, 1))

In [136]:
list_train = [X_train[i:i + 100] for i in range(0, len(X_train), 100)]
list_test = [X_test[i:i + 100] for i in range(0, len(X_test), 100)]
elmo_train = [elmo_vectors(x) for x in list_train]
elmo_test = [elmo_vectors(x) for x in list_test]
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)

ResourceExhaustedError: Graph execution error:

Detected at node 'bilm/CNN/Conv2D_4' defined at (most recent call last):
    File "C:\Users\Austin\anaconda3\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Austin\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\users\austin\downloads\multimodalgenreclassification\env\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\users\austin\downloads\multimodalgenreclassification\env\lib\site-packages\traitlets\config\application.py", line 1053, in launch_instance
      app.start()
    File "c:\users\austin\downloads\multimodalgenreclassification\env\lib\site-packages\ipykernel\kernelapp.py", line 737, in start
      self.io_loop.start()
    File "c:\users\austin\downloads\multimodalgenreclassification\env\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.a

In [8]:
with tf.device('/GPU:0'):
    gpu_lyrics = tf.constant(prep_lyrics)
    embeddings = elmo(gpu_lyrics)['elmo']

ResourceExhaustedError: Graph execution error:

Detected at node 'bilm/embedding_lookup' defined at (most recent call last):
    File "C:\Users\Austin\anaconda3\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Austin\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\users\austin\downloads\multimodalgenreclassification\env\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\users\austin\downloads\multimodalgenreclassification\env\lib\site-packages\traitlets\config\application.py", line 1053, in launch_instance
      app.start()
    File "c:\users\austin\downloads\multimodalgenreclassification\env\lib\site-packages\ipykernel\kernelapp.py", line 737, in start
      self.io_loop.start()
    File "c:\users\austin\downloads\multimodalgenreclassification\env\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      se

In [48]:
trainX = np.reshape(train_tokenized_sentences, (train_tokenized_sentences.shape[0], 1, train_tokenized_sentences.shape[1]))
testX = np.reshape(test_tokenized_sentences, (test_tokenized_sentences.shape[0], 1, test_tokenized_sentences.shape[1]))

### BERT Tokenizer with Dense Net

In [47]:
if MODE_PDUMP:
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    
    train_tokenized_sentences = tokenizer(X_train, padding=True, return_tensors="tf")
    test_tokenized_sentences = tokenizer(X_test, padding=True, return_tensors="tf")
    
    print(train_tokenized_sentences['input_ids'].shape)
    
    train_tokenized_sentences = train_tokenized_sentences['input_ids']
    test_tokenized_sentences = test_tokenized_sentences['input_ids']
    with open('pickles/bert_tok_train.pkl', 'wb') as f:
        pickle.dump(train_tokenized_sentences, f)
    with open('pickles/bert_tok_test.pkl', 'wb') as f:
        pickle.dump(test_tokenized_sentences, f)
else:
    with open('pickles/bert_tok_train.pkl', 'rb') as f:
        train_tokenized_sentences = pickle.load(f)
    with open('pickles/bert_tok_test.pkl', 'rb') as f:
        test_tokenized_sentences = pickle.load(f)

Token indices sequence length is longer than the specified maximum sequence length for this model (629 > 512). Running this sequence through the model will result in indexing errors


(35888, 5184)


In [58]:
# Needed shape for the test tensor
desired_shape = (test_tokenized_sentences.shape[0], train_tokenized_sentences.shape[1])
num_columns_to_add = desired_shape[1] - test_tokenized_sentences.shape[1]
print(desired_shape)
zeros_to_add = tf.zeros((desired_shape[0], num_columns_to_add), dtype=test_tokenized_sentences.dtype)
resulting_tensor_x_test = tf.concat([test_tokenized_sentences, zeros_to_add], axis=1)

(8972, 5184)


In [57]:
resulting_tensor_x_test

<tf.Tensor: shape=(8972, 5184), dtype=int32, numpy=
array([[ 101, 2028, 2298, ...,    0,    0,    0],
       [ 101, 1999, 1996, ...,    0,    0,    0],
       [ 101, 1045, 2196, ...,    0,    0,    0],
       ...,
       [ 101, 2317, 5610, ...,    0,    0,    0],
       [ 101, 2023, 2086, ...,    0,    0,    0],
       [ 101, 2129, 2146, ...,    0,    0,    0]])>

In [63]:
num_classes = 11

bert_dense = tf.keras.Sequential([
    Dense(2048, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.5),
    Dense(1024, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.5),
    Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.5),
    Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.5),
    Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.5),
    Dense(num_classes, activation='softmax')
])

chosen_opt = tf.keras.optimizers.Adam(1e-4)
# chosen_opt ='adam'
bert_dense.compile(loss='sparse_categorical_crossentropy', optimizer=chosen_opt, metrics=['accuracy'])

In [64]:
bert_dense.fit(train_tokenized_sentences, y_train, validation_data=(resulting_tensor_x_test, y_test), epochs=10, batch_size=32,
               callbacks=[es_cb, getModelCheckpoint("bert_dense")])

Epoch 1/10
1122/1122 [==============================] - 14s 12ms/step - loss: 3.1745 - accuracy: 0.1558 - val_loss: 2.2078 - val_accuracy: 0.3005
Epoch 2/10
1122/1122 [==============================] - 13s 11ms/step - loss: 2.6539 - accuracy: 0.2155 - val_loss: 2.1247 - val_accuracy: 0.3094
Epoch 3/10
1122/1122 [==============================] - 13s 11ms/step - loss: 2.4217 - accuracy: 0.2417 - val_loss: 2.0927 - val_accuracy: 0.3073
Epoch 4/10
1122/1122 [==============================] - 13s 11ms/step - loss: 2.2944 - accuracy: 0.2617 - val_loss: 2.0739 - val_accuracy: 0.3120
Epoch 5/10
1122/1122 [==============================] - 13s 12ms/step - loss: 2.2252 - accuracy: 0.2775 - val_loss: 2.0470 - val_accuracy: 0.3136
Epoch 6/10
1122/1122 [==============================] - 13s 12ms/step - loss: 2.1668 - accuracy: 0.2895 - val_loss: 2.0432 - val_accuracy: 0.3131
Epoch 7/10
1122/1122 [==============================] - 13s 12ms/step - loss: 2.1296 - accuracy: 0.2959 - val_loss: 2.0389 -

In [65]:
l, a = bert_dense.evaluate(resulting_tensor_x_test, y_test, verbose=2)
print(f'Test Accuracy: {a}')

281/281 - 1s - loss: 2.0283 - accuracy: 0.3182 - 1s/epoch - 4ms/step
Test Accuracy: 0.31821221113204956


### LSTM Network with Tokenizer

In [105]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding

In [69]:
t = Tokenizer(oov_token='<UNK>')
t.fit_on_texts(X_train)
t.word_index['<PAD>'] = 0

In [79]:
X_train_tok_lstm = t.texts_to_sequences(X_train)
X_test_tok_lstm = t.texts_to_sequences(X_test)

In [99]:
vocab_size = len(t.word_index)
maxlen = len(max(X_train_tok_lstm, key=len))
emb_dim = 300

In [100]:
X_train_tok_lstm = sequence.pad_sequences(X_train_tok_lstm, maxlen=maxlen)
X_test_tok_lstm = sequence.pad_sequences(X_test_tok_lstm, maxlen=maxlen)

In [116]:
num_classes = 11

tok_lstm = tf.keras.Sequential([
    Embedding(vocab_size, emb_dim, input_length=maxlen),
    LSTM(256, return_sequences=True, dropout=0.3),
    Dropout(0.5),
    LSTM(128, return_sequences=True, dropout=0.3),
    Dropout(0.5),
    LSTM(64, return_sequences=False, dropout=0.3),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])

# chosen_opt = tf.keras.optimizers.Adam(1e-4)
chosen_opt ='adam'
tok_lstm.compile(loss='sparse_categorical_crossentropy', optimizer=chosen_opt, metrics=['accuracy'])

In [118]:
tok_lstm.fit(X_train_tok_lstm, y_train, validation_data=(X_test_tok_lstm, y_test), epochs=10, batch_size=32,
             callbacks=[es_cb, getModelCheckpoint("tok_lstm")])

Epoch 1/10
1122/1122 [==============================] - 490s 435ms/step - loss: 2.1400 - accuracy: 0.2856 - val_loss: 2.1327 - val_accuracy: 0.2846
Epoch 2/10
1122/1122 [==============================] - 501s 447ms/step - loss: 1.9822 - accuracy: 0.3299 - val_loss: 1.9061 - val_accuracy: 0.3488
Epoch 3/10
1122/1122 [==============================] - 508s 453ms/step - loss: 1.8710 - accuracy: 0.3623 - val_loss: 1.8812 - val_accuracy: 0.3561
Epoch 4/10
1122/1122 [==============================] - 506s 451ms/step - loss: 1.7522 - accuracy: 0.3929 - val_loss: 1.8514 - val_accuracy: 0.3648
Epoch 5/10
1122/1122 [==============================] - 504s 450ms/step - loss: 1.6123 - accuracy: 0.4346 - val_loss: 1.8772 - val_accuracy: 0.3749
Epoch 6/10
1122/1122 [==============================] - 507s 452ms/step - loss: 1.4713 - accuracy: 0.4859 - val_loss: 1.9364 - val_accuracy: 0.3591
Epoch 7/10
1122/1122 [==============================] - 512s 456ms/step - loss: 1.3053 - accuracy: 0.5470 - val_

In [95]:
y_test.shape

(8972,)